# 设计Echart版本的ec数据读取形式

In [13]:
class ec_data_point:
    def __init__(self,start_time,end_time):
        self.timelist = [0,2,4,6,8,10,12,14,16,18,20,22,24,25,
                         26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,
                        41,42,43,44,45,46,47,48,49,50,51,52]
        self.test_time = '2022041700'
        self.file_path = "/home/liyuan3970/Data/My_Git/" + self.test_time + "/" 
        self.lat_list = [27.532,27.832,27.532,27.532,27.532,27.532,27.532,27.532,27.532,27.532]
        self.lon_list = [121.932,120.932,121.932,121.932,121.932,121.932,121.932,121.932,121.932,121.932]
        self.name = ['洪家','玉环市','温岭市','椒江区','路桥区','黄岩区','仙居县','三门县','临海市','天台县']
        self.name_en = ['hongjia','yuhuan','wenling','jiaojiang','luqiao','huangyan','xianju','sanmen','linhai','tiantai']
        self.name_village = ['sanmen','linhai','xianju','tiantai','jiaojiang','wenling','yuhuan','luqiao','huangyan']
        self.cp,self.t2,self.lsp = self.read_data()
    # 外部函数
    def transform_from_latlon(self,lat, lon):
        lat = np.asarray(lat)
        lon = np.asarray(lon)
        trans = Affine.translation(lon[0], lat[0])
        scale = Affine.scale(lon[1] - lon[0], lat[1] - lat[0])
        return trans * scale    
    def rasterize(self,shapes, coords, latitude='lat', longitude='lon',fill=np.nan, **kwargs):
        transform = self.transform_from_latlon(coords[latitude], coords[longitude])
        out_shape = (len(coords[latitude]), len(coords[longitude]))
        raster = features.rasterize(shapes, out_shape=out_shape,
                                fill=fill, transform=transform,
                                dtype=float, **kwargs)
        spatial_coords = {latitude: coords[latitude], longitude: coords[longitude]}
        return xr.DataArray(raster, coords=spatial_coords, dims=(latitude, longitude))
    def add_shape_coord_from_data_array(self,xr_da, shp_path, coord_name):   
        shp_gpd = gpd.read_file(shp_path)
        shapes = [(shape, n) for n, shape in enumerate(shp_gpd.geometry)]
        xr_da[coord_name] = self.rasterize(shapes, xr_da.coords, longitude='lon', latitude='lat')
        return xr_da
    # 数据分析
    def time_today(self):
        '''用于计算当前日期'''
        pass
    def regrid(self,data):
        # 插值
        ds_out = xr.Dataset(
            {   
                
                "lat": (["lat"], np.arange(27.0, 30, 0.05)),
                "lon": (["lon"], np.arange(120, 122.5, 0.05)),
            }
        )
        regridder = xe.Regridder(data, ds_out, "bilinear")
        dr_out = regridder(data)
        return dr_out
    def read_data(self):
        '''读取数据'''
        files = os.listdir(self.file_path)
        lsp_list = [] 
        cp_list = [] 
        t2_list = [] 
        for fileitem in self.timelist:
            f=xr.open_dataset(self.file_path +files[fileitem],decode_times=False)
            lsp = f.lsp.sel(lonS=slice(118,123),latS=slice(32,26))
            cp = f.cp.sel(lonS=slice(118,123),latS=slice(32,26))
            t2 = f.t2.sel(lonS=slice(118,123),latS=slice(32,26)) 
            lsp_list.append(lsp)  
            cp_list.append(cp)  
            t2_list.append(t2)
            del f,lsp,cp,t2  
        cp_all = xr.concat(cp_list,dim="time")
        cp_all = self.regrid(cp_all)
        t2_all = xr.concat(t2_list,dim="time")
        t2_all = self.regrid(t2_all)
        lsp_all = xr.concat(lsp_list,dim="time")
        lsp_all = self.regrid(lsp_all)
        return cp_all,t2_all,lsp_all
    def accum_data(self,list_data):
        '''处理累计降水'''
        out_list = []
        for i in range(len(list_data)):
            if i==0:
                out_list.append(0)
            elif i==1:
                out_list.append(round(list_data[i],1))
            else:
                out_list.append(round(list_data[i]-list_data[i-1],1))
        return out_list
    def rain_data(self,lat,lon,data):
        list_data= data.sel(lon=lon, lat=lat,method='nearest').to_pandas().tolist()
        out_list = self.accum_data(list_data)
        return out_list
    def plot_line(self,lat,lon):
        '''返回单点的降水气温曲线图'''
        cp_line = self.rain_data(lat,lon,self.cp)
        lsp_line = self.rain_data(lat,lon,self.lsp)
        t2_line = self.t2.sel(lon=lon, lat=lat,method='nearest').to_pandas().tolist()
        return cp_line,lsp_line,t2_line
    def village_data(self,data):
        data_xr = data
        shp_path = "static/data/shpfile/"
        shp_data = gpd.read_file("static/data/shpfile/xiangzhen/xiangzhen.shp", encoding='utf8')
        village_list = shp_data['NAME'].values
        county_list = shp_data['COUNTY'].values
        shp_da = self.add_shape_coord_from_data_array(data_xr, shp_path+"taizhou_village.shp", "country")
        data_dir = {
            "village":[],
            "county":[],
            "value":[]           
        }
        for i in range(len(village_list)):
            awash_da = shp_da.where(shp_da.country==i, other=np.nan)
            name = village_list[i]
            county = county_list[i]
            if np.isnan(awash_da.mean().values.tolist()):
                data_dir['value'].append(0)
            else:
                data_dir['value'].append(awash_da.mean().values.tolist())
            data_dir['village'].append(name)
            data_dir['county'].append(county)
        village = pd.DataFrame(data_dir)
        return village 
    def group_data(self,data):
        '''排序'''     
        rain = []
        village = []
        grouped = data.groupby('county')
        for i in grouped.size().index:
            village_list = []
            singel_rain = {
                "name":None,
                "value":None
            }
            county = grouped.get_group(i)
            sort_data = county.sort_values('value',ascending=False)[0:10]
            village_data = county.sort_values('value',ascending=False)[0:10]
            singel_rain['name'] = sort_data.county.tolist()[0]
            singel_rain['value']= round(county.value.mean(),2)
            rain.append(singel_rain)
            # 数据
            name = sort_data.county.tolist()[0]
            for j,k in zip(sort_data['village'],sort_data['value']):
                # print(j,k)
                singel_village ={}
                singel_village['value'] = round(k,2)
                singel_village['name'] = j            
                village_list.append(singel_village)
            village.append(village_list)
        return rain,village
    def comput_average(self,start_time,end_time):
        '''计算面雨量'''   
        data = { }
        if start_time==0:
            lsp = self.lsp[end_time,:,:] 
            comput = self.village_data(lsp)
            for i in range(len(self.name)):
                county = {                 
                    "lsp":None,
                    "cp":None,
                    "t2":None,
                    "time":None    
                       }
                cp_line,lsp_line,t2_line = self.plot_line(self.lat_list[i],self.lon_list[i])
                county['lsp'] = lsp_line
                county['cp'] = cp_line
                county['t2'] = t2_line 
                county['time'] = self.timelist
                data[self.name_en[i]] = county
            rain,village= self.group_data(comput) 
            data["rain"] = rain
            data["village"] = village
        else:
            lsp1 = self.lsp[end_time,:,:] 
            lsp0 = self.lsp[start_time,:,:] 
            dif = lsp1 - lsp0 
            comput = self.village_data(dif)
            for i in range(len(self.name)):
                county = {                 
                    "lsp":None,
                    "cp":None,
                    "t2":None,
                    "time":None    
                       }
                cp_line,lsp_line,t2_line = self.plot_line(self.lat_list[i],self.lon_list[i])
                county['lsp'] = lsp_line
                county['cp'] = cp_line
                county['t2'] = t2_line 
                county['time'] = self.timelist
                data[self.name_en[i]] = county
            rain,village= self.group_data(comput) 
            data["rain"] = rain
            data["village"] = village
        return data

/opt/conda/lib/python3.7/site-packages/xarray/core/dataarray.py:784: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return key in self.data


CPU times: user 1.49 s, sys: 23.7 ms, total: 1.52 s
Wall time: 1.83 s


In [6]:
a.keys()

dict_keys(['hongjia', 'yuhuan', 'wenling', 'jiaojiang', 'luqiao', 'huangyan', 'xianju', 'sanmen', 'linhai', 'tiantai', 'rain', 'village'])

In [ ]:
data = {
    "面雨量":[], 
    "临海市"：{
        "line":{
            "lsp":[],
            "cp":[],
            "t2":[],
            "time":[]
        },
        "village":{
            "rain":[],
            "line":[],
            "name":[]          
        }
    },
    
}

In [1]:
import geopandas as gpd


shp_path = "shpfile/xiangzhen/xiangzhen.shp"
shp_gpd = gpd.read_file(shp_path, encoding='utf8')
shp_gpd

/opt/conda/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


,FEATUREGUI,FCODE,NAME,FSCALE,USOURCE,UPDATETIME,UPDATESTAT,SHAPE_Leng,SHAPE_Area,COUNTY,SHI,TYPE,CITYCODE,geometry
0,3dd51852d9dc44f5ab5cb0004bd766bf,6601000500,安洲街道,13,20190402基础库全量包,20190402,U,0.542087,0.004292,仙居县,台州市,4,None,"POLYGON ((120.68685 28.91842, 120.68606 28.914..."
1,8425d006aeb34b858d86120247526bef,6601000500,桃渚镇,13,20190402基础库全量包,20190402,U,1.018298,0.011036,临海市,台州市,4,None,"MULTIPOLYGON (((121.67986 28.79381, 121.67952 ..."
2,03b8f0e160754d608c5747b467d7e5c8,6601000500,蓬街镇,13,20190402基础库全量包,20190402,U,0.511900,0.006772,路桥区,台州市,4,None,"MULTIPOLYGON (((121.60227 28.55373, 121.60217 ..."
3,094b7e3bcc3144329512958c9d6fc8be,6601000500,杜桥镇,13,20190402基础库全量包,20190402,U,0.900278,0.016811,临海市,台州市,4,None,"POLYGON ((121.47616 28.82388, 121.47818 28.823..."
4,91d31aae17484169a101b846138f620c,6601000500,海门街道,13,2020年6月基础库政区数据,20200611,U,0.302182,0.002189,椒江区,台州市,4,None,"POLYGON ((121.50494 28.66132, 121.49950 28.658..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,5b8d3b8c1b044a2c886155549a6d753c,6601000500,大陈镇,13,2020年6月基础库政区数据,20200611,A,1.231533,0.001371,椒江区,台州市,4,None,"MULTIPOLYGON (((121.90887 28.39064, 121.90949 ..."
127,ab5f6c265fba43559d1cf7838a08a727,6601000500,城南镇,13,2020年6月基础库政区数据,20200611,A,1.055532,0.010046,温岭市,台州市,4,None,"MULTIPOLYGON (((121.43654 28.26580, 121.43835 ..."
128,566991a69fc24be5845186e4678c735a,6601000500,花桥镇,13,2020年6月基础库政区数据,20200611,A,0.569683,0.006595,三门县,台州市,4,None,"MULTIPOLYGON (((121.56132 28.93053, 121.56174 ..."
129,d19857af684e40f0ad5b03a2690f2765,6601000500,新桥镇,13,20190402基础库全量包,20190402,U,0.233467,0.001272,路桥区,台州市,4,None,"POLYGON ((121.42574 28.53229, 121.43003 28.525..."


# Leaflet版本的ec数据

In [1]:
%%time
import geopandas as gpd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from scipy.interpolate import interp1d
import xesmf as xe
import redis
import pandas as pd
import datetime
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
import os
import random
from io import BytesIO
import base64
from rasterio import features
from affine import Affine



class ec_data_point:
    def __init__(self,start_time,end_time):
        self.timelist = [0,2,4,6,8,10,12,14,16,18,20,22,24,25,
                         26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,
                        41,42,43,44,45,46,47,48,49,50,51,52]
        self.test_time = '2022041700'
        self.file_path = "/workspace/liyuan3970/Data/My_Git/" + self.test_time + "/" 
        self.lat_list = [27.6, 28.1, 28.4, 29.1, 29.1, 29.8, 28.7, 28.5, 28.5, 28.6]
        self.lon_list = [120.7, 121.1, 121.3, 121.2, 121.0, 120.7, 121.1, 121.4, 121.4, 121.2]
        self.name = ["台州", "玉环", "温岭", "三门", "天台", "仙居", "临海", "路桥", "椒江", "黄岩"]
        self.name_en = ['taizhou','yuhuan','wenling','sanmen','tiantai','xianju','linhai','luqiao','jiaojiang','huangyan']
        self.read_data()
        #self.cp,self.t2,self.lsp = self.read_data()
    # 外部函数
    def transform_from_latlon(self,lat, lon):
        lat = np.asarray(lat)
        lon = np.asarray(lon)
        trans = Affine.translation(lon[0], lat[0])
        scale = Affine.scale(lon[1] - lon[0], lat[1] - lat[0])
        return trans * scale    
    def rasterize(self,shapes, coords, latitude='lat', longitude='lon',fill=np.nan, **kwargs):
        transform = self.transform_from_latlon(coords[latitude], coords[longitude])
        out_shape = (len(coords[latitude]), len(coords[longitude]))
        raster = features.rasterize(shapes, out_shape=out_shape,
                                fill=fill, transform=transform,
                                dtype=float, **kwargs)
        spatial_coords = {latitude: coords[latitude], longitude: coords[longitude]}
        return xr.DataArray(raster, coords=spatial_coords, dims=(latitude, longitude))
    def add_shape_coord_from_data_array(self,xr_da, shp_path, coord_name):   
        shp_gpd = gpd.read_file(shp_path)
        shapes = [(shape, n) for n, shape in enumerate(shp_gpd.geometry)]
        xr_da[coord_name] = self.rasterize(shapes, xr_da.coords, longitude='lon', latitude='lat')
        return xr_da
    def regrid(self,data):
        # 插值
        ds_out = xr.Dataset(
            {   
                
                "lat": (["lat"], np.arange(27.0, 30, 0.05)),
                "lon": (["lon"], np.arange(120, 122.5, 0.05)),
            }
        )
        regridder = xe.Regridder(data, ds_out, "bilinear")
        dr_out = regridder(data)
        return dr_out
    # 读取数据
    def read_data(self):
        '''读取数据'''
        files = os.listdir(self.file_path)
        lsp_list = [] 
        cp_list = [] 
        t2_list = [] 
        f=xr.open_dataset("/workspace/liyuan3970/Data/My_Git/2022041700/ecfine.I2022041700.000.F2022041700.nc",decode_times=False)
#         for fileitem in self.timelist:
#             datapath = self.file_path + str(files[fileitem])
#             print("数据",type(fileitem),type(self.file_path),type(files[fileitem]),type(datapath),datapath)           
#             f=xr.open_dataset(datapath,decode_times=False)
#             lsp = f.lsp.sel(lonS=slice(118,123),latS=slice(32,26))
#             #lsp = lsp.swap_dims({'latS':'lat','lonS':'lon'})
#             cp = f.cp.sel(lonS=slice(118,123),latS=slice(32,26))
#             #cp = cp.swap_dims({'latS':'lat','lonS':'lon'})
#             t2 = f.t2.sel(lonS=slice(118,123),latS=slice(32,26)) 
#             #t2 = t2.swap_dims({'latS':'lat','lonS':'lon'})
#             lsp_list.append(lsp)  
#             cp_list.append(cp)  
#             t2_list.append(t2)
#             del f,lsp,cp,t2  
#         cp_all = xr.concat(cp_list,dim="time")
#         #cp_all = self.regrid(cp_all)
#         t2_all = xr.concat(t2_list,dim="time")
#         #t2_all = self.regrid(t2_all)
#         lsp_all = xr.concat(lsp_list,dim="time")
#         #lsp_all = self.regrid(lsp_all)
#         return cp_all,t2_all,lsp_all
    # 曲线的读取
    def accum_data(self,list_data):
        '''处理累计降水'''
        out_list = []
        for i in range(len(list_data)):
            if i==0:
                out_list.append(0)
            elif i==1:
                out_list.append(round(list_data[i],1))
            else:
                out_list.append(round(list_data[i]-list_data[i-1],1))
        return out_list
    def rain_data(self,lat,lon,data):
        list_data= data.sel(lon=lon, lat=lat,method='nearest').to_pandas().tolist()
        out_list = self.accum_data(list_data)
        return out_list
    def plot_line(self,lat,lon):
        '''返回单点的降水气温曲线图'''
        cp_line = self.rain_data(lat,lon,self.cp)
        totle_line = self.rain_data(lat,lon,self.lsp)
        lsp_line = []
        for i in range(len(totle_line)):
            lsp_line.append(totle_line[i] - cp_line[i])
        t2_line = self.t2.sel(lon=lon, lat=lat,method='nearest').to_pandas().tolist()
        return cp_line,lsp_line,t2_line
    def conuty_data(self):
        '''将数据整理成json并存储到MySQL'''
        data_list = []
        for i in range(len(self.lat_list)):
            lat = self.lat_list[i]
            lon = self.lon_list[i]
            cp_line,lsp_line,t2_line = self.plot_line(lat,lon)
            data_single = {
                "name":self.name_en[i],
                "cp":cp_line,
                "lsp":lsp_line,
                "t2":t2_line
            }
            data_list.append(data_single)
        return data_list
    # 定时任务
    def to_sql(self):
        '''将数据传到sql中'''
        pass
    # 截面数据的读取
    def rander_leaflet(self,start_time,end_time):
        '''返回数据'''
        if start_time==0:
            rain = end_rain 
        else:
            start_rain = self.lsp[start_time,:,:]
            end_rain = self.lsp[end_time,:,:]
            rain = end_rain - start_rain 
        filepath = "static/data/shpfile/"
        shp_da = self.add_shape_coord_from_data_array(rain, filepath+"taizhou.shp", "remain") 
        taizhou = shp_da.where(shp_da.remain<7, other=0.0)
        len_lat = len(taizhou.lat.data)
        len_lon = len(taizhou.lon.data)
        data = []
        for i in range(len_lon-1):
            for j in range(len_lat-1):
                #y0 = round(27.00+j*0.05,2)
                #x0 = round(120.00+i*0.05,2)
                y0 = taizhou.lat.data[j]
                x0 = taizhou.lon.data[i]
                if taizhou.data[j, i]!=0:
                    # single = {
                    #     "Lat":y0,
                    #     "Lon":x0,
                    #     "value":str(taizhou.data[j, i])
                    # }
                    single = {
                        "type": "Feature",
                        "properties": {
                            "value": str(taizhou.data[j, i])
                        },
                        "geometry": {
                            "type": "Point",
                            "coordinates": [x0, y0]
                        }
                    }
                    data.append(single)  
        return data
    
start_time,end_time = 1,24
ec_worker = ec_data_point(start_time,end_time) 
# ec_worker.rander_leaflet(start_time,end_time) 
# ec_worker.plot_line(27.532,121.932)
# ec_worker.rander_leaflet(start_time,end_time)
# a = ec_worker.comput_average(start_time,end_time)


/opt/conda/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


TypeError: expected string or bytes-like object

In [7]:
import xarray as xr
f=xr.open_dataset("TZ_self/20220401/I20220401080000.Pr01.nc",decode_times=False)

TypeError: expected string or bytes-like object

In [3]:
ec_worker.cp

<xarray.DataArray (time: 41, lat: 60, lon: 50)>
array([[[          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        ...,
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan]],

       [[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
...
        [6.9407783e+00, 6.4130440e+00, 5.8857985e+00, ...,
         4.7254655e-01, 4.0662104e-01, 4.5056763e-01],
        [7.3253598e+00, 6.7248130e+00, 6.1240396e+00, ...,
         3.9187646e-01, 3.5889715e-01, 5.2732444e-01],
        [6.5928240e+00, 6.2559905e+00, 5.9189296e+00, ...,
         5.5303431e-01, 6.0063392e-01, 8.8626689e-01]],

       [[5.2651028e+01, 5.4922379e+01, 5.7193703e+01, ...,
         4.0162216e+01, 4.0143833e+01, 4.0766552e+01],
        [5.3749832e+01, 5.3632969e+01, 5.3516068e+01, ...,
         3.8344917e+01, 3.8099560e+01, 3.7931465e+01],
        [5.4848637e+01, 5.2343704e+01, 4.9838741e+01, ...,
         3.6528233e+01, 3.6055786e+01, 3.5096554e+01],
        ...,
        [6.9407783e+00, 6.4789681e+00, 6.0176497e+00, ...,
         4.7254655e-01, 4.0662104e-01, 4.5056763e-01],
        [7.3253598e+00, 6.7907453e+00, 6.2559018e+00, ...,
         3.9187646e-01, 3.5889715e-01, 5.2732444e-01],
        [6.5928240e+00, 6.3072720e+00, 6.0214906e+00, ...,
         5.5303431e-01, 6.0063392e-01, 8.8626689e-01]]], dtype=float32)
Coordinates:
  * time     (time) int64 0 6 12 18 24 30 36 42 ... 204 210 216 222 228 234 240
  * lon      (lon) float64 120.0 120.0 120.1 120.1 ... 122.3 122.3 122.4 122.4
  * lat      (lat) float64 27.0 27.05 27.1 27.15 27.2 ... 29.8 29.85 29.9 29.95
Attributes:
    regrid_method:  bilinear

# 测试xarray


In [2]:
import xarray as xr
f=xr.open_dataset("TZ_self/20220402/I20220401080000.Pr01.nc")

AttributeError: module 'dask.array' has no attribute 'lib'